# Set-Up

Installations and imports:

In [ ]:
import os
from math import ceil
import time
import json
import numpy as np
import sys
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import fenics_helpers

Plotting settings:

In [ ]:
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
mpl.rcParams['figure.dpi'] = 100
mpl_save_dpi = 300

Save settings:

In [ ]:
fig_save_dir = './figs'
if not os.path.isdir(fig_save_dir):
    os.mkdir(fig_save_dir)
data_save_dir = './data'
if not os.path.isdir(data_save_dir):
    os.mkdir(data_save_dir)

Directory of breast mesh:

In [ ]:
BREAST_DIR = './breast_closed.obj'

Define parameters which never change between simulations:

In [ ]:
always_fixed_params = {# Physical parameters:
                       'density': 0.00102, # in g/mm^3
                       'g': 9.81, # in m/s^3
                       'elem_order': 2,
                       # Non-linear solver parameters:
                       'num_load_steps': 30, 
                       'rtol': 1e-9, 
                       'atol': 1e-9, 
                       'max_iter': 50,
                       'kappa': 3000} # in kPa   

# Convergence Study

## Mesh Size Convergence

In [ ]:
fixed_params = {'C_1': 1.9, # in kPa
                'y_rot': 90, # in degrees
                'x_rot': 0, # in degrees
                **always_fixed_params}

# Vary number of surface points on mesh:
num_surfpts_list = [25, 50, 100, 250, 500, 750, 1000]

results = {key: [] for key in ('max_disp', 'num_surfpts', 'num_elem', 'volume')}
for i, num_surfpts in enumerate(num_surfpts_list):
    mesh = create_breast_mesh(BREAST_DIR, num_surfpts)
    num_elem = mesh.geometry.dofmap.array.shape[0]
    print(f"Simulating Mesh {i+1}/{len(num_surfpts_list)} (Num elem = {num_elem})")
    
    u = fenics_helpers.simulate_neohookean_breast(mesh, **fixed_params)
    
    results['num_surfpts'].append(num_surfpts)
    results['num_elem'].append(num_elem)
    
results['fixed_params'] = fixed_params

with open(os.path.join(data_save_dir, 'neohookean_breast_meshsize_convergence.json'), 'w') as f:
    json.dump(results, f, indent=4)